미세조정 전후 비교

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

# 미세 조정전 모델(랜덤 초기화)
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_before = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME)
model_before.eval()  #평가모드
with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_before(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()

        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정 : {probs[0]} | 부정 : {probs[1]}\n')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


리뷰 : 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 부정 0.6838129162788391
긍정 : 0.3161871135234833 | 부정 : 0.6838129162788391

리뷰 : 2 : Terrible waste of time. Worst movie ever made.
예측 : 부정 0.6747531890869141
긍정 : 0.32524681091308594 | 부정 : 0.6747531890869141

리뷰 : 3 : It was okay, nothing special but not bad either.
예측 : 부정 0.6742779016494751
긍정 : 0.3257220983505249 | 부정 : 0.6742779016494751

리뷰 : 4 : Brilliant acting and stunning visuals throughout!
예측 : 부정 0.6586832404136658
긍정 : 0.34131672978401184 | 부정 : 0.6586832404136658

리뷰 : 5 : Boring and predictable plot. Very disappointed.
예측 : 부정 0.6899210214614868
긍정 : 0.31007900834083557 | 부정 : 0.6899210214614868



In [4]:
# 학습 - 미세조정
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
train_texts = [
    ("This is wonderful and fantastic!", 1),  # 긍정
    ("Absolutely terrible and awful!", 0),     # 부정
    ("I love this so much!", 1),
    ("Hated it completely!", 0)
]
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_after = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME,num_labels=2)
from transformers import AdamW
optimizer = AdamW(model_after.parameters(), lr=2e-5)
model_after.train()
for epoch in range(7):
    total_loss = 0
    for text,label in train_texts:
        optimizer.zero_grad()
        inputs = tokenizer(text,return_tensors='pt')
        labels = torch.tensor([label])
        outputs = model_after(**inputs,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_texts)
    print(f'epoch : {epoch+1}  loss : {avg_loss:.4f}')    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch : 1  loss : 0.6921
epoch : 2  loss : 0.4813
epoch : 3  loss : 0.3664
epoch : 4  loss : 0.3324
epoch : 5  loss : 0.2071
epoch : 6  loss : 0.1784
epoch : 7  loss : 0.1466


In [5]:
print(f'미세조정 후')
test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_after(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()        
        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정 : {probs[0]} | 부정 : {probs[1]}\n')

미세조정 후
리뷰 : 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 부정 0.7521082758903503
긍정 : 0.24789169430732727 | 부정 : 0.7521082758903503

리뷰 : 2 : Terrible waste of time. Worst movie ever made.
예측 : 긍정 0.8100009560585022
긍정 : 0.8100009560585022 | 부정 : 0.1899990439414978

리뷰 : 3 : It was okay, nothing special but not bad either.
예측 : 긍정 0.6539788842201233
긍정 : 0.6539788842201233 | 부정 : 0.3460211753845215

리뷰 : 4 : Brilliant acting and stunning visuals throughout!
예측 : 부정 0.7070790529251099
긍정 : 0.29292088747024536 | 부정 : 0.7070790529251099

리뷰 : 5 : Boring and predictable plot. Very disappointed.
예측 : 긍정 0.7239748239517212
긍정 : 0.7239748239517212 | 부정 : 0.2760251760482788

